<a href="https://colab.research.google.com/github/DavidSchineis/Math-Physics/blob/main/Copy_of_Lab_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Abstract
This lab focuses on verifying the Divergence Theorem. We used ConvexHull to construct triangular meshes for different closed surfaces such as spheres, half-spheres, cylinders, cubes, and cones. We used meshgrid to create surface integrals of various vector fields. Then, we set up volume integrals by calculating divergence with np.gradient, masking out anything outside of our bounds. For each case, we compared the two inetegral results and confirmed that they matched closely with each other as well as with the by-hand value. This lab highlights how surface and volume integrals can be implemented in Python and how they can be used to confirm the Divergence Theorem across different shapes. Importantly, we also showed that the results follow the expected dependence on radius, height, or side length for each shape, and matched the by-hand calculations when at the given radius, height, or length.

---

As the midway point through the semester, today's lab will serve as a culmination of all the various skills that we have practiced throughout this semester as we numerically verify the divergence theorem. However, setting up surface integrals from scratch is far from trivial, thus the code how to do so is provided to you below. This code can take an arbitrary array of triangles that define each small surface on a 3-d mesh, and finds the integral of a vector along the surface of this triangle.

In [ ]:
import numpy as np
from numpy import pi, cos, sin, tan, cosh, sinh, tanh
from scipy.spatial import ConvexHull
import plotly.graph_objects as go

def surface_integral_from_triangulation(vertices,vector):
  def surface_triangle(a, b, c, resolution=20):
    ab = b - a
    ac = c - a
    normal = np.cross(ab, ac).astype(np.float64)
    normal /= np.linalg.norm(normal)


    u = np.linspace(0, 1, resolution)
    v = np.linspace(0, 1, resolution)
    uu, vv = np.meshgrid(u, v)
    mask = (uu + vv <= 1)

    points = a[None, None, :] + uu[..., None]*ab + vv[..., None]*ac
    xs, ys, zs = points[..., 0], points[..., 1], points[..., 2]

    return normal, xs, ys, zs, mask

  ar = 0
  for i in range(len(vertices)):
    a, b, c = vertices[i, 0], vertices[i, 1], vertices[i, 2]
    normal, xs, ys, zs, mask = surface_triangle(a, b, c)

    center = np.mean(p, axis=0)
    if np.dot(normal, a - center) < 0:
        normal = -normal

    vxs, vys, vzs = vector(xs, ys, zs)
    dot = (normal[0] * vxs + normal[1] * vys + normal[2] * vzs)

    triangle_area = 0.5 * np.linalg.norm(np.cross(b - a, c - a))
    area = triangle_area * np.mean(dot[mask])
    ar += area

  return(ar)

Let's evaluate this integral along the surface of a sphere. Similarly to how we have done in a few labs back, define x, y, and z positions corresponding to the surface of the sphere with radius of 1, and make a 3d plot of it. In defining theta and phi angles, don't use more than 30 points to aid visualization.

In [ ]:
theta = np.linspace(0, 2*pi, 30)
phi   = np.linspace(0, pi, 30)

thetaGrid, phiGrid = np.meshgrid(theta, phi, indexing='xy')

x = sin(phiGrid) * cos(thetaGrid)
y = sin(phiGrid) * sin(thetaGrid)
z = cos(phiGrid)

fig = go.Figure()
fig.add_trace(go.Surface(x=x, y=y, z=z, showscale=False))

fig.update_layout(
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z'
    ),
    scene_aspectmode='data'
)

fig.show()

### Caption
This is a surface plot of a unit sphere in spherical coordinates where r=1, theta=0 to 2pi, phi=0 to pi.


Having all of these individual points at hand, we will stack x, y, and z positions into a single array, and then, using ConvexHull, for each point we will find neighboring points that could be used as the vertices of trinangles.

In [ ]:
p = np.stack([x.flatten(), y.flatten(), z.flatten()]).T


tri = ConvexHull(p)
indices = tri.simplices
vertices = p[indices]
xs=x.flatten()[indices]
ys=y.flatten()[indices]
zs=z.flatten()[indices]

print(xs)


To visualize the resulting mesh that this has created, create an empty plotly figure, and then, iterating over all of the triangles that were created above in a for loop, add all of these triangles to the plot using go.Scatter3d function.

In [ ]:
fig = go.Figure()

for i in range(len(vertices)):

  xtri = [xs[i,0],xs[i,1],xs[i,2],xs[i,0]]
  ytri = [ys[i,0],ys[i,1],ys[i,2],ys[i,0]]
  ztri = [zs[i,0],zs[i,1],zs[i,2],zs[i,0]]

  fig.add_trace(go.Scatter3d(x=xtri, y=ytri, z=ztri,mode='lines'))

fig.show()

### Caption
This a surface plot of unit sphere made with trianglular mesh in spherical coordinates where r=1, theta=0 to 2pi, phi=0 to pi.

We will now define a vector

$\vec{v}=z\hat{z}$

Using this vector, we will compute the surface integral using the provided function

In [ ]:
def vector(xx,yy,zz):
    vx=np.zeros_like(xx)
    vy=np.zeros_like(yy)
    vz=zz
    return(vx,vy,vz)

ar=surface_integral_from_triangulation(vertices,vector)
print("Computed surface integral:", ar)



We will now perform a test to see if the surface integral of the divergence of this vector matches the surface integral we have found.

- To do that, first define x, y, and z arrays to define our axes (all to be between -1 and 1), then pass them along to meshgrid (don't forget indexing='ij') to define xyz positions of every single point within a cube.

- Calculate vx, vy, and vz components of the vector for every single point in this cube.

- Using np.gradient, calculate the divergence within every single point of the cube.

- Similarly to how we approached it last time, using np.where, find points that are outside the surface of our sphere with radius 1, and set the divergence for these points to zero, to make them not be a factor while we are doing the volume integral.

- Finally a volume integral would be very similar to the surface integral we have done last time. First take the divergence and integrate it with respect to z, then take that result and integrate it with respect to y, finally, integrate the result with respect to x.

In [ ]:
x,y,z=np.linspace(-1,1),np.linspace(-1,1),np.linspace(-1,1)
xx,yy,zz = np.meshgrid(x,y,z, indexing='ij')

vx = np.zeros_like(xx)
vy = np.zeros_like(yy)
vz = zz

div= (np.gradient(vx,x,axis=0) + np.gradient(vy,y,axis=1) + np.gradient(vz,z,axis=2))

a = np.where(np.sqrt((xx**2) + (yy**2) + (zz**2)) > 1)
div[a] = 0

V = np.trapezoid(div,z,axis=2)
V = np.trapezoid(V,y,axis=1)
V = np.trapezoid(V,x,axis=0)

print(V)


#### Question:
- How closely were you able to match two sets of integrals to each other?
- Evaluate either the surface or the volume integral yourself on paper. What is the expected value? Write out a full expression, to explicitly show the dependence on radius, and then evaluate it for r=1. How closely were you able to recover it?
#### Answer:
- The surafce integral was 4.143913573412861 while the volume integral was 4.177307074433272.
- The expected by-hand value is 4pir^3/3 which at r=1 is 4pi/3 = 4.189. Our calculated integrals are within 1-2% error of the by-hand value

----
Repeat the above (including visualizations), but for a half-sphere with radius of 1 above the x-y plane with $\vec{v}=r\hat{z}$

In [ ]:
theta = np.linspace(0, 2*pi, 30)
phi   = np.linspace(0, pi/2, 30)

thetaGrid, phiGrid = np.meshgrid(theta, phi, indexing='xy')

x = sin(phiGrid) * cos(thetaGrid)
y = sin(phiGrid) * sin(thetaGrid)
z = cos(phiGrid)

fig = go.Figure()
fig.add_trace(go.Surface(x=x, y=y, z=z, showscale=False))

fig.update_layout(
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z'
    ),
    scene_aspectmode='data'
)

fig.show()



p = np.stack([x.flatten(), y.flatten(), z.flatten()]).T

tri = ConvexHull(p)
indices = tri.simplices
vertices = p[indices]
xs=x.flatten()[indices]
ys=y.flatten()[indices]
zs=z.flatten()[indices]



fig = go.Figure()

for i in range(len(vertices)):

  xtri = [xs[i,0],xs[i,1],xs[i,2],xs[i,0]]
  ytri = [ys[i,0],ys[i,1],ys[i,2],ys[i,0]]
  ztri = [zs[i,0],zs[i,1],zs[i,2],zs[i,0]]

  fig.add_trace(go.Scatter3d(x=xtri, y=ytri, z=ztri,mode='lines'))

fig.show()



def vector(xx,yy,zz):
    r = np.sqrt(xx**2 + yy**2 + zz**2)
    vx=np.zeros_like(xx)
    vy=np.zeros_like(yy)
    vz=r
    return(vx,vy,vz)

ar=surface_integral_from_triangulation(vertices,vector)
print("Computed surface integral:", ar)



x,y,z=np.linspace(-1,1),np.linspace(-1,1),np.linspace(0,1)
xx,yy,zz = np.meshgrid(x,y,z, indexing='ij')

vx = np.zeros_like(xx)
vy = np.zeros_like(yy)
vz = np.sqrt((xx**2) + (yy**2) + (zz**2))

div= (np.gradient(vx,x,axis=0) + np.gradient(vy,y,axis=1) + np.gradient(vz,z,axis=2))

r = np.sqrt((xx**2) + (yy**2) + (zz**2))
a = np.where(r > 1)
div[a] = 0

V = np.trapezoid(div,z,axis=2)
V = np.trapezoid(V,y,axis=1)
V = np.trapezoid(V,x,axis=0)

print(V)

### Caption
First plot is a surface plot of a half unit sphere in spherical coordinates where r=1, theta=0 to 2pi, phi=0 to pi/2.
Second plot is a surface plot of a half unit sphere made with trianglular mesh in spherical coordinates where r=1, theta=0 to 2pi, phi=0 to pi/2.

#### Question:
- How closely were you able to match two sets of integrals to each other?
- Evaluate either the surface or the volume integral yourself on paper. What is the expected value? Write out a full expression, to explicitly show the dependence on radius, and then evaluate it for r=1. How closely were you able to recover it?
#### Answer:
- The surafce integral was 0.985978133826052 while the volume integral was 1.045366901291943.
- The expected by-hand value is pir^3/3 which at r=1 is pi/3 = 1.047. Our calculated integrals are within 0-6% error of the by-hand value with the surface integral varying much more than the volume.

----
Repeat the above (including visualizations), but for a cylinder with radius of 2, and height from 0 to 3 and the vector field of $\vec{v}=r\hat{r}$.

In [ ]:
R = 2
theta = np.linspace(0, 2*pi, 30)
z = np.linspace(0, 3, 30)

thetaGrid, zGrid = np.meshgrid(theta, z, indexing='xy')

x = 2*cos(thetaGrid)
y = 2*sin(thetaGrid)
z = zGrid

fig = go.Figure()
fig.add_trace(go.Surface(x=x, y=y, z=z, showscale=False))

fig.update_layout(
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z'
    ),
    scene_aspectmode='data'
)

fig.show()



p = np.stack([x.flatten(), y.flatten(), z.flatten()]).T

tri = ConvexHull(p)
indices = tri.simplices
vertices = p[indices]
xs=x.flatten()[indices]
ys=y.flatten()[indices]
zs=z.flatten()[indices]



fig = go.Figure()

for i in range(len(vertices)):

  xtri = [xs[i,0],xs[i,1],xs[i,2],xs[i,0]]
  ytri = [ys[i,0],ys[i,1],ys[i,2],ys[i,0]]
  ztri = [zs[i,0],zs[i,1],zs[i,2],zs[i,0]]

  fig.add_trace(go.Scatter3d(x=xtri, y=ytri, z=ztri, mode='lines'))

fig.show()



def vector(xx,yy,zz):
    vx=xx
    vy=yy
    vz=np.zeros_like(zz)
    return(vx,vy,vz)

ar=surface_integral_from_triangulation(vertices,vector)
print("Computed surface integral:", ar)



x,y,z=np.linspace(-R,R),np.linspace(-R,R),np.linspace(0,3)
xx,yy,zz = np.meshgrid(x,y,z, indexing='ij')

vx = xx
vy = yy
vz = np.zeros_like(zz)

div= (np.gradient(vx,x,axis=0) + np.gradient(vy,y,axis=1) + np.gradient(vz,z,axis=2))

r = np.sqrt(xx**2 + yy**2)
a = np.where(r > R)
div[a] = 0

V = np.trapezoid(div,z,axis=2)
V = np.trapezoid(V,y,axis=1)
V = np.trapezoid(V,x,axis=0)

print(V)

### Caption

First plot is a surface plot of a cyllinder in cyllindrical coordinates where r=2, theta=0 to 2pi, z=0 to 3. Second plot is a surface plot of a cyllinder made with trianglular mesh in cyllindrical coordinates where r=2, theta=0 to 2pi, z=0 to 3.

#### Question:
- How closely were you able to match two sets of integrals to each other?
- Evaluate either the surface or the volume integral yourself on paper. What is the expected value? Write out a full expression, to explicitly show the dependence on radius and height, and then evaluate it for r=2 and h=3. How closely were you able to recover it?
#### Answer:
- The surafce integral was 74.8097131934363 while the volume integral was 75.00874635568512.
- The expected by-hand value is 2pir^2h which at r=2 and h=3 is 24pi = 75.398. Both integrals are within 1% error of the by-hand value which is great agreement.

----
Repeat the above (including the mesh visualization, but not the surface one), but for a cube which has edges between 0 and 1 and the vector field of $\vec{v}=xyz\hat{z}+xyz\hat{y}+xyz\hat{z}$.

Hint: we do not have a fancy function to create a cube in the same way we have for a sphere or a cylinder, but we can define all of the verticies of a cube by hand, there are 8 of them in total.

In [ ]:
p =np.array([[0,0,0],[1,0,0],[0,1,0],[1,1,0],[0,0,1],[1,0,1],[0,1,1],[1,1,1]])
tri = ConvexHull(p)
indices = tri.simplices
vertices = p[indices]
xs = p[:,0].flatten()[indices]
ys = p[:,1].flatten()[indices]
zs = p[:,2].flatten()[indices]



fig = go.Figure()

for i in range(len(vertices)):

  xtri = [xs[i,0],xs[i,1],xs[i,2],xs[i,0]]
  ytri = [ys[i,0],ys[i,1],ys[i,2],ys[i,0]]
  ztri = [zs[i,0],zs[i,1],zs[i,2],zs[i,0]]

  fig.add_trace(go.Scatter3d(x=xtri, y=ytri, z=ztri,mode='lines'))

fig.show()



def vector(xx,yy,zz):
    vx=xx*yy*zz
    vy=xx*yy*zz
    vz=xx*yy*zz
    return(vx,vy,vz)

ar=surface_integral_from_triangulation(vertices,vector)
print("Computed surface integral:", ar)



x,y,z=np.linspace(0,1),np.linspace(0,1),np.linspace(0,1)
xx,yy,zz = np.meshgrid(x,y,z, indexing='ij')

vx = xx*yy*zz
vy = xx*yy*zz
vz = xx*yy*zz

div= (np.gradient(vx,x,axis=0) + np.gradient(vy,y,axis=1) + np.gradient(vz,z,axis=2))

r = np.sqrt(xx**2 + yy**2)
a = np.where(r > R)
div[a] = 0

V = np.trapezoid(div,z,axis=2)
V = np.trapezoid(V,y,axis=1)
V = np.trapezoid(V,x,axis=0)

print(V)

### Caption
This is a surface plot of a cube made with the triangulated frame of a unit cube going from 0 to 1 in (x, y, z) built from its 8 vertices using ConvexHull.

#### Question:
- How closely were you able to match two sets of integrals to each other?
- Evaluate either the surface or the volume integral yourself on paper. What is the expected value? Write out a full expression, to explicitly show the dependence on the side of the cube, and then evaluate it for b=1. How closely were you able to recover it?
#### Answer:
- The surafce integral was 0.7368421052631579 while the volume integral was 0.7500000000000001.
- The expected by-hand value is 3b^5/4 which at b=1 is 3/4 = 0.75. Both integrals are within 2% error of the by-hand value with the volume integral getting extremely close.

---
# Extra credit

Repeat the above (including visualizations), but for a cone that satisfies a surface of $x^2+y^2=z^2$, with z ranging between 0 and 1, and the vector field of $\vec{v}=(x-y)\hat{x}+(x+z)\hat{y}+(z-y)\hat{z}$.

In [ ]:
theta = np.linspace(0, 2*pi, 30)
z = np.linspace(0, 1, 30)

thetaGrid, zGrid = np.meshgrid(theta, z, indexing='xy')

x = zGrid*cos(thetaGrid)
y = zGrid*sin(thetaGrid)
z = zGrid

fig = go.Figure()
fig.add_trace(go.Surface(x=x, y=y, z=z, showscale=False))

fig.update_layout(
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z'
    ),
    scene_aspectmode='data'
)

fig.show()



p = np.stack([x.flatten(), y.flatten(), z.flatten()]).T

tri = ConvexHull(p)
indices = tri.simplices
vertices = p[indices]
xs=x.flatten()[indices]
ys=y.flatten()[indices]
zs=z.flatten()[indices]



fig = go.Figure()

for i in range(len(vertices)):

  xtri = [xs[i,0],xs[i,1],xs[i,2],xs[i,0]]
  ytri = [ys[i,0],ys[i,1],ys[i,2],ys[i,0]]
  ztri = [zs[i,0],zs[i,1],zs[i,2],zs[i,0]]

  fig.add_trace(go.Scatter3d(x=xtri, y=ytri, z=ztri,mode='lines'))

fig.show()



def vector(xx,yy,zz):
    vx=xx-yy
    vy=xx+zz
    vz=zz-yy
    return(vx,vy,vz)

ar=surface_integral_from_triangulation(vertices,vector)
print("Computed surface integral:", ar)



x,y,z=np.linspace(-1,1),np.linspace(-1,1),np.linspace(0,1)
xx,yy,zz = np.meshgrid(x,y,z, indexing='ij')

vx = xx-yy
vy = xx+zz
vz = zz-yy

div= (np.gradient(vx,x,axis=0) + np.gradient(vy,y,axis=1) + np.gradient(vz,z,axis=2))

a = np.where((xx**2 + yy**2) > zz**2)
div[a] = 0

V = np.trapezoid(div,z,axis=2)
V = np.trapezoid(V,y,axis=1)
V = np.trapezoid(V,x,axis=0)

print(V)

### Caption
First plot is a surface plot of a cone in cartesian coordinates defined by x^2 + y^2 = z^2 where z=0 to 1. Second plot is a surface plot of the same cone made with trianglular mesh in cartesian coordinates defined by x^2 + y^2 = z^2 where z=0 to 1.

#### Question:
- How closely were you able to match two sets of integrals to each other?
- Evaluate either the surface or the volume integral yourself on paper. What is the expected value? Write out a full expression, to explicitly show the dependence on height, and then evaluate it for z=1. How closely were you able to recover it?
#### Answer:
- The surafce integral was 2.078047588706566 while the volume integral was 2.0828056337070437.
- The expected by-hand value is 2pih^3/3 which at h=1 is 2pi/3 = 2.094. Both integrals are within 1% error of the by-hand value maintaining good agreement.